In [1]:
# this python script calcualtes annual mean aridity and spei index from GRIDMET for a specified timeperiod
# for a specified shape files
# we are using geemap python module for GEE (https://geemap.org/)
# credit is given for geemap developer
# you need to first install geemap package in your python environment
# before running this script
# to do this, we need shapefiles in wgs84 format


Enter verification code:  4/1ASVgi3J88smcu-XS7N3EaWvnn80AbDzLmUuoTrPqB_FJuO6x4ISyVLlMxlA



Successfully saved authorization token.


In [ ]:
# authenticate 
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [2]:
#import modules
import geemap
import os


In [154]:
# read shapefile 
# since we are reading shape files for each 250 polygons
# 250 polygons were choosen because when choosing more than that, GEE send error for excedding computational resources
# we are defining values for reading shape files


# provide path for your specified file
shp_file = 'C:\\Users\\..\\..\\filenamd.shp'

# Example
str_idx = 0
end_idx = 250
shp_file = 'C:\\Users\\..\\..\\..\\all_LTDL_WRI_projects_wgs_first_'+ str(str_idx) +'_to_'+str(end_idx)+'.shp'



# convert each shapefile inside the shape file to GEE feature
polys = geemap.shp_to_ee(shp_file)


# now read image collections for aridity index calculation
coll_gridmet =  ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')

# select start and end year
srt_yr = 1980
end_yr = 2020

# filter data based on date range, band, and feature
precip = coll_gridmet.select('pr')
et = coll_gridmet.select('eto')


# define a function to calcualte aridity index (ai)
def ai(image):
    image = image.addBands(image.expression(
            '(precip/(et))',
        {
                'precip': image.select('pr'),
                'et': image.select('eto'),
                 }).rename('ai'))\
                .copyProperties(image, ['system:time_start'])

    return image

coll_ai = coll.map(ai)     

# now calcualte annual mean ai
# define a function that calculates yearly mean

years = ee.List.sequence(srt_yr, end_yr)

def yr_mean_ai(year):
    image = coll_ai.filter(ee.Filter.calendarRange(year, year, 'year')).select('ai').mean().rename('yr_mean_ai')
    return image

# map this function over all image colelction to year yearly mean ai
coll_ai_yr_mean = coll_ai.fromImages(years.map(yr_mean_ai))



# for spei
# filter based on date range, band, and feature

coll_gridmet_drought =  ee.ImageCollection('GRIDMET/DROUGHT')
band = ['spei1y']

coll_spei = coll_gridmet_drought.select(band)

def yr_mean_spei(year):
    image = coll_spei.filter(ee.Filter.calendarRange(year, year, 'year')).select('spei1y').mean().rename('yr_mean_spei')
    return image

# map this function over all image colelction
coll_spei_yr_mean = coll_spei.fromImages(years.map(yr_mean_spei))

# now export ai data
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')



# we export data when scale = 4000m and 1000m
# when exporting with scale=4000m, we are missing some values
# here scale = 4000 is choosen to match ~4km of GRIDMET resolution
# possibly because polygons are very small, but after changing scale=1000m, we have values for all polygons, may be due downscaled value
#export ai
out_ai_stats = os.path.join(out_dir, 'GRIDMET_Annual_AI_FOR_DART_'+ str(str_idx) +'_to_'+str(end_idx)+'.csv')
geemap.zonal_statistics(coll_ai_yr_mean.select('yr_mean_ai'), polys, out_ai_stats, statistics_type = 'MEAN', scale=1000)


# export spei
out_spei_stats = os.path.join(out_dir, 'GRIDMET_Annual_SPEI_FOR_DART_'+ str(str_idx) +'_to_'+str(end_idx)+'.csv')
geemap.zonal_statistics(coll_spei_yr_mean.select('yr_mean_spei'), polys, out_spei_stats, statistics_type = 'MEAN', scale=1000)



###################################
# end

###################################



Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to C:\Users\sdhital\Downloads\GRIDMET_Annual_AI_FOR_DART_1400_to_1600.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to C:\Users\sdhital\Downloads\GRIDMET_Annual_SPEI_FOR_DART_1400_to_1600.csv


In [141]:
# you can also check data and features by plotting them
Map = geemap.Map()
Map.addLayer(polys, {}, 'polys')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…